In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as md
import calendar
from calendar import monthrange

%matplotlib inline 

In [8]:
df = pd.read_csv('prod_anal3.csv')

In [9]:
df.dtypes

date             object
event            object
purchase_sum    float64
os_name          object
device_id         int64
gender           object
city             object
utm_source       object
dtype: object

### Вычислить MAU февраля

In [10]:
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.strftime('%m/%Y')

In [11]:
df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,01/2020
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,01/2020
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,01/2020
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,01/2020
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,01/2020
...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,03/2020
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,03/2020
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,03/2020
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,03/2020


In [12]:
df.query('month=="02/2020"').agg({'device_id': 'nunique'})

device_id    75032
dtype: int64

### Количество установок в январе:

In [13]:
df.query('month=="01/2020" & event=="app_install"').agg({'device_id': 'count'})

device_id    80297
dtype: int64

### Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? Ответ впишите в формате: дд.мм.гггг

In [14]:
df1= df[['date', 'event', 'device_id']]
df1

,date,event,device_id
0,2020-01-01,app_start,669460
1,2020-01-01,app_start,833621
2,2020-01-01,app_start,1579237
3,2020-01-01,app_start,1737182
4,2020-01-01,app_start,4029024
...,...,...,...
2747963,2020-03-31,register,2984778
2747964,2020-03-31,register,27301864
2747965,2020-03-31,register,1294285
2747966,2020-03-31,register,3010574


In [15]:
df_date_install = df1.query('event=="app_install"').groupby('device_id', as_index=False).agg({'date': 'min'})
df_date_install.columns = ['device_id', 'installation_date']
df_date_install

,device_id,installation_date
0,4013,2020-01-15
1,4016,2020-01-04
2,4046,2020-01-04
3,4065,2020-03-19
4,4067,2020-01-31
...,...,...
154592,35379281,2020-03-29
154593,35380796,2020-03-31
154594,35381595,2020-03-30
154595,35388218,2020-03-31


In [23]:
df_new = df.merge(df_date_install, on = 'device_id', how = 'left')
df_new

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,installation_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,01/2020,2020-01-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,01/2020,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,01/2020,NaT
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,01/2020,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,01/2020,2020-01-01
...,...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,03/2020,2020-03-28
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,03/2020,2020-03-31
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,03/2020,2020-03-31
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,03/2020,2020-03-06


In [17]:
df_new1 = df_new.query('event=="purchase"')
df_new1

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,installation_date
7691,2020-01-01,purchase,265.5,android,6081155,female,Moscow,-,01/2020,2020-01-01
7692,2020-01-01,purchase,374.0,android,77448,female,Saint-Petersburg,yandex-direct,01/2020,2020-01-01
7693,2020-01-01,purchase,98.5,ios,7458607,female,Moscow,instagram_ads,01/2020,2020-01-01
7694,2020-01-01,purchase,304.5,ios,3520947,male,Moscow,instagram_ads,01/2020,2020-01-01
7695,2020-01-01,purchase,153.5,android,1170605,male,Moscow,google_ads,01/2020,2020-01-01
...,...,...,...,...,...,...,...,...,...,...
2507469,2020-03-31,purchase,6546.5,android,11864116,male,Saint-Petersburg,google_ads,03/2020,2020-03-25
2507470,2020-03-31,purchase,3495.0,android,3403969,female,Saint-Petersburg,google_ads,03/2020,NaT
2507471,2020-03-31,purchase,2857.5,ios,26120362,female,Moscow,google_ads,03/2020,2020-03-31
2507472,2020-03-31,purchase,5671.5,ios,22011351,female,Moscow,yandex-direct,03/2020,2020-01-10


In [18]:
df_new1['diff'] = df_new1['date'] - df_new1['installation_date']
df_new1

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,installation_date,diff
7691,2020-01-01,purchase,265.5,android,6081155,female,Moscow,-,01/2020,2020-01-01,0 days
7692,2020-01-01,purchase,374.0,android,77448,female,Saint-Petersburg,yandex-direct,01/2020,2020-01-01,0 days
7693,2020-01-01,purchase,98.5,ios,7458607,female,Moscow,instagram_ads,01/2020,2020-01-01,0 days
7694,2020-01-01,purchase,304.5,ios,3520947,male,Moscow,instagram_ads,01/2020,2020-01-01,0 days
7695,2020-01-01,purchase,153.5,android,1170605,male,Moscow,google_ads,01/2020,2020-01-01,0 days
...,...,...,...,...,...,...,...,...,...,...,...
2507469,2020-03-31,purchase,6546.5,android,11864116,male,Saint-Petersburg,google_ads,03/2020,2020-03-25,6 days
2507470,2020-03-31,purchase,3495.0,android,3403969,female,Saint-Petersburg,google_ads,03/2020,NaT,NaT
2507471,2020-03-31,purchase,2857.5,ios,26120362,female,Moscow,google_ads,03/2020,2020-03-31,0 days
2507472,2020-03-31,purchase,5671.5,ios,22011351,female,Moscow,yandex-direct,03/2020,2020-01-10,81 days


In [19]:
df_new2 = df_new1.query('diff <="7 days"')
df_new2

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,installation_date,diff
7691,2020-01-01,purchase,265.5,android,6081155,female,Moscow,-,01/2020,2020-01-01,0 days
7692,2020-01-01,purchase,374.0,android,77448,female,Saint-Petersburg,yandex-direct,01/2020,2020-01-01,0 days
7693,2020-01-01,purchase,98.5,ios,7458607,female,Moscow,instagram_ads,01/2020,2020-01-01,0 days
7694,2020-01-01,purchase,304.5,ios,3520947,male,Moscow,instagram_ads,01/2020,2020-01-01,0 days
7695,2020-01-01,purchase,153.5,android,1170605,male,Moscow,google_ads,01/2020,2020-01-01,0 days
...,...,...,...,...,...,...,...,...,...,...,...
2507458,2020-03-31,purchase,5029.0,android,30148472,male,Moscow,vk_ads,03/2020,2020-03-31,0 days
2507463,2020-03-31,purchase,2861.0,ios,8700073,male,Moscow,vk_ads,03/2020,2020-03-31,0 days
2507467,2020-03-31,purchase,6948.0,android,13278240,female,Moscow,instagram_ads,03/2020,2020-03-30,1 days
2507469,2020-03-31,purchase,6546.5,android,11864116,male,Saint-Petersburg,google_ads,03/2020,2020-03-25,6 days


In [33]:
df_purchases = df_new2.groupby('installation_date', as_index=False).agg({'event': 'count'})
df_purchases['date'] = df_purchases['installation_date']
df_purchases

,installation_date,event,date
0,2020-01-01,1932,2020-01-01
1,2020-01-02,1546,2020-01-02
2,2020-01-03,1043,2020-01-03
3,2020-01-04,799,2020-01-04
4,2020-01-05,728,2020-01-05
...,...,...,...
86,2020-03-27,288,2020-03-27
87,2020-03-28,219,2020-03-28
88,2020-03-29,236,2020-03-29
89,2020-03-30,207,2020-03-30


In [31]:
df_guests = df_new.groupby('date', as_index=False).agg({'device_id': 'nunique'})
df_guests

,date,device_id
0,2020-01-01,4445
1,2020-01-02,5104
2,2020-01-03,5144
3,2020-01-04,4925
4,2020-01-05,5608
...,...,...
86,2020-03-27,7810
87,2020-03-28,6469
88,2020-03-29,5935
89,2020-03-30,5705


In [39]:
df_final = df_purchases.merge(df_guests, on = 'date', how = 'left')
df_final.columns = ['installation_date', 'purchases', 'date', 'visitors']
df_final

,installation_date,purchases,date,visitors
0,2020-01-01,1932,2020-01-01,4445
1,2020-01-02,1546,2020-01-02,5104
2,2020-01-03,1043,2020-01-03,5144
3,2020-01-04,799,2020-01-04,4925
4,2020-01-05,728,2020-01-05,5608
...,...,...,...,...
86,2020-03-27,288,2020-03-27,7810
87,2020-03-28,219,2020-03-28,6469
88,2020-03-29,236,2020-03-29,5935
89,2020-03-30,207,2020-03-30,5705


In [41]:
df_final['rolling_purchases'] = df_final['purchases'].rolling(7).sum()
df_final['rolling_visitors'] = df_final['visitors'].rolling(7).sum()
df_final

,installation_date,purchases,date,visitors,rolling_purchases,rolling_visitors
0,2020-01-01,1932,2020-01-01,4445,NaN,NaN
1,2020-01-02,1546,2020-01-02,5104,NaN,NaN
2,2020-01-03,1043,2020-01-03,5144,NaN,NaN
3,2020-01-04,799,2020-01-04,4925,NaN,NaN
4,2020-01-05,728,2020-01-05,5608,NaN,NaN
...,...,...,...,...,...,...
86,2020-03-27,288,2020-03-27,7810,2470.0,58894.0
87,2020-03-28,219,2020-03-28,6469,2410.0,58519.0
88,2020-03-29,236,2020-03-29,5935,2360.0,57753.0
89,2020-03-30,207,2020-03-30,5705,2135.0,54654.0


In [51]:
df_final['conversion'] = df_final['rolling_purchases']/df_final['rolling_visitors']*100
df_final.head(25)

,installation_date,purchases,date,visitors,rolling_purchases,rolling_visitors,conversion
0,2020-01-01,1932,2020-01-01,4445,NaN,NaN,NaN
1,2020-01-02,1546,2020-01-02,5104,NaN,NaN,NaN
2,2020-01-03,1043,2020-01-03,5144,NaN,NaN,NaN
3,2020-01-04,799,2020-01-04,4925,NaN,NaN,NaN
4,2020-01-05,728,2020-01-05,5608,NaN,NaN,NaN
5,2020-01-06,612,2020-01-06,5165,NaN,NaN,NaN
6,2020-01-07,514,2020-01-07,4774,7174.0,35165.0,20.400967
7,2020-01-08,504,2020-01-08,4951,5746.0,35671.0,16.108323
8,2020-01-09,747,2020-01-09,5417,4947.0,35984.0,13.747777
9,2020-01-10,5287,2020-01-10,21431,9191.0,52271.0,17.583364


In [50]:
df_final.sort_values('conversion', ascending=False)

,installation_date,purchases,date,visitors,rolling_purchases,rolling_visitors,conversion
6,2020-01-07,514,2020-01-07,4774,7174.0,35165.0,20.400967
9,2020-01-10,5287,2020-01-10,21431,9191.0,52271.0,17.583364
14,2020-01-15,1933,2020-01-15,11858,15033.0,88528.0,16.981068
15,2020-01-16,1437,2020-01-16,10635,15723.0,93746.0,16.771916
13,2020-01-14,2266,2020-01-14,12943,13604.0,81621.0,16.667279
...,...,...,...,...,...,...,...
1,2020-01-02,1546,2020-01-02,5104,NaN,NaN,NaN
2,2020-01-03,1043,2020-01-03,5144,NaN,NaN,NaN
3,2020-01-04,799,2020-01-04,4925,NaN,NaN,NaN
4,2020-01-05,728,2020-01-05,5608,NaN,NaN,NaN
